# Group S
 
- António Carvalho - m20200642 
- Frederico Rodrigues - m20200583 
- Gonçalo Carvalho - m20200664 
- Manuel Borges - 

In [1]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [9]:
import pandas as pd
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, KFold, cross_validate, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.metrics import accuracy_score

In [3]:
%store -r datafe
datafe

,LeadTime,ArrivalDateDayOfMonth,StaysInWeekendNights,StaysInWeekNights,Adults,Children,Babies,PreviousCancellations,PreviousBookingsNotCanceled,BookingChanges,...,x10_ 484,x10_ 495,x10_ 509,x10_ 0,x11_Group,x11_Transient,x11_Transient-Party,x12_1,x12_2,x12_3
0,6,1,0,2,1,0.0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,88,1,0,4,2,0.0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,65,1,0,4,1,0.0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,92,1,2,4,2,0.0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,100,2,0,2,2,0.0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79325,23,30,2,5,2,0.0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
79326,102,31,2,5,3,0.0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
79327,34,31,2,5,2,0.0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
79328,109,31,2,5,2,0.0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [4]:
%store -r treshold1
%store -r treshold2
%store -r onlyDT
%store -r target

treshold1 = treshold1.tolist()
treshold2 = treshold2.tolist()

In [5]:
#heavily correlated variables
onlyDT.remove('ADRpp')
onlyDT.remove('TotalNights')

## Modeling

In [6]:
X = datafe[onlyDT]
y = target

# Provisional TrainTest Split
#X_train, X_val, y_train, y_val = train_test_split(data[final_var],
                                                  #target, test_size=0.2, random_state=15, stratify=target)

In [10]:
models = [("LR_model", LogisticRegression(random_state=42,n_jobs=-1)),
        ("DT_model", DecisionTreeClassifier(random_state=42)),
        ("RF_model", RandomForestClassifier(random_state=42,n_jobs=-1)),
        ("SVC:model", SVC(kernel="rbf", C=0.025, probability=True)),
        ("NuSVC:model", NuSVC(probability=True)),
        ("MLP_model", MLPClassifier(hidden_layer_sizes=(150,100,50), max_iter=300,activation = 'relu',random_state=1)),
        ("XGB_model", XGBClassifier(random_state=42, n_jobs=-1))]

#split data for cross-validation
split = KFold(n_splits=4, shuffle=True, random_state=42)

In [11]:
for name, model in models:
    # cross validation score for each model:
    cv_results = cross_val_score(model, 
                                 X, y,
                                 cv=split,
                                 scoring="accuracy",
                                 n_jobs=-1)
    # output:
    min_score = round(min(cv_results), 3)
    max_score = round(max(cv_results), 3)
    mean_score = round(np.mean(cv_results), 3)
    std_dev = round(np.std(cv_results), 3)
    print(f"{name} cross validation accuarcy score: {mean_score} +/- {std_dev} (std) min: {min_score}, max: {max_score}")

LR_model cross validation accuarcy score: 0.774 +/- 0.006 (std) min: 0.765, max: 0.781
DT_model cross validation accuarcy score: 0.818 +/- 0.002 (std) min: 0.816, max: 0.822
RF_model cross validation accuarcy score: 0.861 +/- 0.001 (std) min: 0.86, max: 0.862
SVC:model cross validation accuarcy score: 0.646 +/- 0.001 (std) min: 0.644, max: 0.647
NuSVC:model cross validation accuarcy score: 0.793 +/- 0.005 (std) min: 0.787, max: 0.8
MLP_model cross validation accuarcy score: 0.83 +/- 0.004 (std) min: 0.824, max: 0.834
XGB_model cross validation accuarcy score: 0.841 +/- 0.002 (std) min: 0.838, max: 0.843
